# Capítulo 1 - Introdução ao Python.

Nada de novo aqui. Avance para o próximo capítulo.

# Capítulo 2 - Pentest em Redes.

## Ciclo básico do pentest:
Coleta de Informações -> Mapeamento de Vulnerabilidades -> Exploração de Falhas -> Escalonamento de Privilégios -> Manutenção do Acesso

* Coleta de Informações: Levantamento de todas as informações sobre o alvo.
    - Registros de DNS
    - Mapeamento de Serviços
    - Coleta de Email
* Mapeamento de Vulnerabilidades: Verifica se os serviços encontrados possuem vulnerabilidades que podem ser exploradas para o ganho de acesso ao ambiente.
    - Scans de Vulnerabilidades
* Exploração de Falhas: Ataca as vulnerabilidades encontradas no ambiente para ganhar acesso ao sistema.
    - Exploits
    - Engenharia Social
* Escalonamento de Privilégios: Uma vez dentro do ambiente, conseguir acesso privilegiado aos sistemas como root ou admin.
* Manutenção do Acesso: Instalação de scripts para permitir o acesso a qualquer tempo aos sistemas comprometidos.

> No livro está bem confusa a descrição dos passos, então eu mudei um pouco pra ficar mais claro.

## 2.1 - Coleta de Informações:
### 2.1.1 - Enumeração de servidores DNS

In [16]:
import socket
dominio = 'dominio.com'
nomes = ['ns1','ns2','www','ftp','intranet']

for nome in nomes:
    dns = nome + '.' + dominio
    try:
        ip = socket.gethostbyname(dns)
        print('{0}: {1}'.format(dns,ip))
    except socket.gaierror:
        pass

ns1.dominio.com: 65.254.244.180
ns2.dominio.com: 65.254.244.180
www.dominio.com: 65.254.244.176
ftp.dominio.com: 65.254.238.128
intranet.dominio.com: 65.254.244.180


O script acima utiliza o método `gethostbyname()` da biblioteca `socket` para determinar o endereço IP de uma lista de servidores através de seus nomes. Caso o nome seja encontrado, a função retorna uma string contendo o endereço IP retornado pelo DNS ou caso contrário, dispara uma exceção do tipo  `gaierror` - *Get Address Information* que retorna uma descrição do erro ocorrido.

*Obs*:
 - O método `gethostbyname()` não suporta IPv6, para verificar endereços de rede IPv6 é necessário usar a função `getaddrinfo()` da biblioteca `socket`.
 - Caso o host possua vários IPs associados ao seu nome, usar a função `gethostbyname()_ex` para conseguir a listagem de todos os endereços associados.

In [18]:
import socket
dominio = 'google.com'

with open('d:/Users/afukaya/OneDrive/Repos/Github/Books/Python para Pentest/Brute-Force.txt') as arquivo:
    nomes = arquivo.readlines()
    
for nome in nomes:
    dns = nome.strip('\n') + '.' + dominio
    try:
        ip = socket.gethostbyname(dns)
    except socket.gaierror:
        ip = 'Não encontrado'
        
    print('{0}: {1}'.format(dns,ip))

ns1.google.com: 216.239.32.10
ns2.google.com: 216.239.34.10
www.google.com: 172.217.28.68
ftp.google.com: Não encontrado
intranet.google.com: Não encontrado
xpto.google.com: Não encontrado


O script anterior foi alterado para ler nome dos hosts a partir de um arquivo text ao invés de uma lista, além disso, alterei o código para exibir a mensagem `Não encontrado` ao lado do nome do host, quando o script não for capaz de encontrar o endereço IP da máquina no DNS.

In [19]:
from queue import Queue
import threading
import socket

dominio = 'dominio.com'
lock = threading.Lock()

def forca_bruta():
    dns = q.get() + '.' + dominio
    try:
        ip = socket.gethostbyname(dns)
        lock.acquire()
        print('{}:\t{}'.format(dns,ip))
    except socket.gaierror:
        pass
    else:
        lock.release()
    q.task_done()
    
q = Queue()
for i in range(20):
    t = threading.Thread(target = forca_bruta)
    t.daemon = True
    t.start()
with open('d:/Users/afukaya/OneDrive/Repos/Github/Books/Python para Pentest/Brute-Force.txt') as lista:
    while True:
        nome = lista.readline().strip('\n')
        if not nome :
            break
        q.put(nome)
    q.join()
    print('Mapeamento completo')

ns1.dominio.com:	65.254.244.180
ns2.dominio.com:	65.254.244.180
www.dominio.com:	65.254.244.176
ftp.dominio.com:	65.254.238.128
intranet.dominio.com:	65.254.244.180
xpto.dominio.com:	65.254.244.180
Mapeamento completo


O script anterior foi modificado para permitir o uso de mutiplas threads na coleta de infomações dos hosts no DNS, acelerando o processo de coleta quanto temos um grande volume de hosts para consultar.
Ele usa os recursos da biblioteca `Queue` para criar uma fila onde os nomes dos hosts serão armazenados e da biblioteca `threading` que permite que a função `forca_bruta()` seja executada em threads paralelas.

> O script é bem interessante, mas o código está confuso. Acredito que ele merece uma repaginada, pois essa estrutura pode ser utilizada em outras atividades que demandam grande volume de processamento como port scanning e network sweep.

In [1]:
import socket
ip = "65.254.244.180"
hostmanes = socket.gethostbyaddr(ip)
print(hostmanes)

('65-254-244-180.yourhostingaccount.com', [], ['65.254.244.180'])


O script acima retorna informações de um host a partir de seu endereço IP. A função `gethostbyadd()` retorna uma tripla contendo o hostname, uma lista de alias e uma lista de IPs para o host informado. Assim como o `gethostbyname()` a função `gethostbyadd()` não funciona com o IPv6.

In [3]:
import dns.resolver
dominio = 'dominio.com'
registros = ['A','AAAA','MX','NS']
for registro in registros:
    resposta = dns.resolver.query(dominio,registro,raise_on_no_answer=False)
    if resposta.rrset is not None:
        print(resposta.rrset)

dominio.com. 3600 IN A 65.254.244.176
dominio.com. 3600 IN MX 30 mx.dominio.com.
dominio.com. 3600 IN NS ns1.accountsupport.com.
dominio.com. 3600 IN NS ns2.accountsupport.com.


O script acima retorna informações sobre os servidores associados aos registros de DNS definidos em `registros` ele usa as funcionalidades da biblioteca `dsnpython` para pesquisar o DNS por informações.

> A biblioteca `dsnpython` não é padrão no python3 e precisa ser instalada para que o script funcione.

## 2.1.2 Transferência de zona

In [6]:
import dns.query
import dns.zone
import dns.resolver

dominio = 'dominio.com'
registroNS = dns.resolver.query(dominio,'NS')
lista = []
for registro in registroNS:
    lista.append(str(registros))

for registro in lista:
    try:
        transferenciaZona = dns.zone.from_xfr(dns.query.xfr(registro,dominio))
    except:
        print('Erro na transferência.')
    else:
        registroDNS = transferenciaZona.nodes.keys()
        registroDNS.sort()
        for n in registroDNS:
            print(transferenciaZona[n].to_text(n))

Erro na transferência.
Erro na transferência.


O script anterior tenta realiza a transferência da zona de DNS de um domínio determinado. Ela lista todos os servidores de DNS do domínio e depois tenta fazer a transferência a partir desses servidores.

> Apesar de confuso para mim, por causa dos nomes das variáveis, o script é interessante para testar se os servidores de DNS do domínio são vulneráveis ao XFER.

Todo: Instalar uma VM com um servidor de DNS que permita o XFER para testar o script.

## 2.1.3 Whois
O script a seguir depende da biblioteca `python-whois` que deve ser instalada.

> A biblioteca `python-whois` não existe no repositório do anaconda e precisa ser instalada a partir do PIP - `pip install python-whois`

In [9]:
import whois
dominio = 'dominio.com'
consultaWhois = whois.whois(dominio)
print(consultaWhois.email)
print(consultaWhois["email"])
print(consultaWhois.text)

None
   Domain Name: DOMINIO.COM
   Registry Domain ID: 433178_DOMAIN_COM-VRSN
   Registrar WHOIS Server: whois.domain.com
   Registrar URL: http://www.domain.com
   Updated Date: 2019-07-23T03:47:23Z
   Creation Date: 1997-08-22T04:00:00Z
   Registry Expiry Date: 2020-08-21T04:00:00Z
   Registrar: Domain.com, LLC
   Registrar IANA ID: 886
   Registrar Abuse Contact Email: compliance@domain-inc.net
   Registrar Abuse Contact Phone: 602-226-2389
   Domain Status: clientDeleteProhibited https://icann.org/epp#clientDeleteProhibited
   Domain Status: clientTransferProhibited https://icann.org/epp#clientTransferProhibited
   Domain Status: clientUpdateProhibited https://icann.org/epp#clientUpdateProhibited
   Name Server: NS1.ACCOUNTSUPPORT.COM
   Name Server: NS2.ACCOUNTSUPPORT.COM
   DNSSEC: unsigned
   URL of the ICANN Whois Inaccuracy Complaint Form: https://www.icann.org/wicf/
>>> Last update of whois database: 2020-01-10T15:38:19Z <<<

For more information on Whois status codes, pleas

O script recupera as informações de registro do domínio especificado.

> O comando `print(consultaWhois["email"])` dá erro caso o campo especificado não exista.

## 2.1.4 Enumeração de usuários via SMTP


In [ ]:
import socket
usuarios = ['daniel','teste','root']
ip = '127.0.0.1'
for usuario in usuarios:
    s = socket.socket(socket.AF_INET,socket.SOCK_STREAM)
    s.connect(ip,25)
    banner = s.recv(1024)
    print(banner)
    s.send('VRFY'+ usuario + '\n')
    resposta = s.recv(1024)
    s.close()
    if '252' in resposta:
        print(usuarios + ' encontrado')
    elif '550' in resposta:
        print(usuarios + ' não encontrado')
    elif '503' in resposta:
        print('Autenticação requerida')
        break
    elif '500' in resposta:
        print('Comando não suportado')
        break
    else:
        print(reposta)

O script acima estabelece uma conexão direta via socket com o servidor SMTP e envia o comando `VRFY` que testa se um usuário existe na base de usuários ou não, testando os códigos de resposta do servidor para determinar o resultado da operação.

> A lista com a descrição dos diversos códigos de erro pode ser encontrada em https://en.wikipedia.org/wiki/List_of_SMTP_server_return_codes

Todo: Instalar um servidor SMTP para testar o script.

## 2.1.5 Scapy

Scapy é uma biblioteca que permite a manipulação de pacotes de rede. Ela pode ser utlizada de maneira iterativa, executando o comando scapy no prompt ou em um script, importando o módulo scapy.

> Ver o que faz os comandos 
    - conf
    - sr


### 2.1.5.1 Port Scanner

In [8]:
from scapy.all import *

conf.verb = 0
portas = [21,22,23,80,8080]
pacoteIP = IP(dst='192.168.0.1')
pacoteTCP = TCP(dport=portas,flags='S')
pacote = pacoteIP/pacoteTCP
ans, uans = sr(pacote, inter=0.1, timeout=1)
print('Porta \t Estado')
for pacoteRecebido in ans:
    print('{} \t {}'.format(pacoteRecebido[1][IP].sport,pacoteRecebido[1][TCP].sprintf('%flags%')))

Porta 	 Estado
21 	 RA
80 	 SA
8080 	 RA


O script anterior tenta se conectar ao IP especificado em `pacoteIP` e testa se as portas especificadas em `portas` e retorna se elas estão abertas `SA` ou fechadas `RA` através do envio de um pacote TCP com a flag `SYN` ativada.`

> Tem outro script mais pra frente que faz a mesma coisa só que com sockets.

### 2.5.1.2 Ping sweep

In [1]:
from scapy.all import *
conf.verb = 0
IPs = []
for ip in range(0,255):
    IPs.append('192.168.0.' + str(ip))
pacote = IP(dst=IPs)/ICMP()
ans,uans = sr(pacote,inter=0.1,timeout=1)
print('Hosts Ativos')
for pacoteRecebido in ans:
    print(pacoteRecebido[1][IP].src)
print('Hosts Inativos')
for pacoteNaoRecebido in uans:
    print(pacoteNaoRecebido.dst)
    

Hosts Ativos
Hosts Inativos
192.168.0.0
192.168.0.1
192.168.0.2
192.168.0.3
192.168.0.4
192.168.0.5
192.168.0.6
192.168.0.7
192.168.0.8
192.168.0.9
192.168.0.10
192.168.0.11
192.168.0.12
192.168.0.13
192.168.0.14
192.168.0.15
192.168.0.16
192.168.0.17
192.168.0.18
192.168.0.19
192.168.0.20
192.168.0.21
192.168.0.22
192.168.0.23
192.168.0.24
192.168.0.25
192.168.0.26
192.168.0.27
192.168.0.28
192.168.0.29
192.168.0.30
192.168.0.31
192.168.0.32
192.168.0.33
192.168.0.34
192.168.0.35
192.168.0.36
192.168.0.37
192.168.0.38
192.168.0.39
192.168.0.40
192.168.0.41
192.168.0.42
192.168.0.43
192.168.0.44
192.168.0.45
192.168.0.46
192.168.0.47
192.168.0.48
192.168.0.49
192.168.0.50
192.168.0.51
192.168.0.52
192.168.0.53
192.168.0.54
192.168.0.55
192.168.0.56
192.168.0.57
192.168.0.58
192.168.0.59
192.168.0.60
192.168.0.61
192.168.0.62
192.168.0.63
192.168.0.64
192.168.0.65
192.168.0.66
192.168.0.67
192.168.0.68
192.168.0.69
192.168.0.70
192.168.0.71
192.168.0.72
192.168.0.73
192.168.0.74
192.168

O script envia uma série de pacotes ICPM - echo request para uma faixa de endereços IP especificada em IPs. Exibindo uma mensagem para todos os hosts que responderam ao request.

### 2.5.1.3 ARP Scanner

In [3]:
from scapy.all import *
conf.verb = 0

IPs = []
for ip in range(0,255):
    IPs.append('192.168.0.' + str(ip))
pacoteARP = Ether()/ARP(pdst=IPs,hwdst='ff:ff:ff:ff:ff:ff')
ans,uans = srp(pacoteARP,inter=0.1,timeout=1)
print('IP \t\t MAC')
for pacoteRecebido in ans:
    print('{} \t\t {}'.format(pacoteRecebido[1].psrc,pacoteRecebido[1].hwscrc))

IP 		 MAC


### 2.5.1.4 HPing

In [ ]:
from scapy.all import *
conf.verb = 0
pacoteIP = IP(dst='192.168.0.1')

# ICMP
ICMPechoRequest = pacoteIP/ICMP()
ICMPechoReply = pacoteIP/ICMP(type=0,code=0)
ICMPtimeStampRequest = pacoteIP/ICMP(type=13,code=0)
ICMPpayload = pacoteIP/ICMP()/'Payload'

# Pacote TCP com SYN flag
pacoteTCP = pacoteIP/TCP(flags='S')

# Pacote TCP com SYN flag, na porta de origem '666' e payload
pacoteTCP1 = pacoteIP/TCP(sport=666)/'Payload'

# Pacote TCP com SYN flag, para a porta 80
pacoteTCP2 = pacoteIP/TCP(dport=80)

# Pacote TCP com SYN flag, para as portas 23 e 80
pacoteTCP3 = pacoteIP/TCP(dport=[23,80])

# Pacote TCP com SYN flag, para porta 80 dos IPs 192.168.0.1  e 192.168.0.2
pacoteTCP4 = IP(dst=['192.168.0.1','192.168.0.2'])/TCP(dport=80)

# Pacote TCP com ACK flag
pacoteTCP5 = pacoteIP/TCP(flags='A')

# Pacote TCP com SYN/ACK flag
pacoteTCP6 = pacoteIP/TCP(flags='SA')

# Pacote IP com IP spofado
pacoteTCP7 = IP(src='1.2.3.4')/TCP()

# Pacote UDP para porta 53
pacoteUDP = pacoteIP/UDP(dport=53)

# Pacote UDP com destino 53 e origem 666
pacoteUDP1 = pacoteIP/UDP(dport=53,sport=666)

### 2.1.5.5 Determinando regras de firewall

In [19]:
from scapy.all import *
conf.verb = 0
host = '192.168.0.1'
portas = [22,80,666,12345]
pacote = IP(dst=host)/TCP(dport=portas,flags='S')
ans,uans = sr(pacote,inter=0.1,timeout=1)

print('Porta \t Estado')

for pacoteRecebido in ans:
    if pacoteRecebido[1].haslayer('ICMP'):
        if pacoteRecebido[1]['ICMP'].type == 3 and pacoteRecebido[1]['ICMP'].code == 3:
            print('{} \t Rejected'.format(pacoteRecebido[0][TCP].dport))
    elif pacoteRecebido[1].haslayer('TCP'):
        print('{} \t {}'.format(pacoteRecebido[1][TCP].sport,pacoteRecebido[1][TCP].sprintf('%flags%')))

for pacoteNaoRecebido in uans:
    print('{} \t DROP'.format(pacoteNaoRecebido.dport))

Porta 	 Estado
80 	 SA
666 	 RA
22 	 DROP
12345 	 DROP


## 2.1.6 Coleta de banner

In [14]:
import socket
s = socket.socket(socket.AF_INET,socket.SOCK_STREAM)
s.connect(('192.168.0.1',80))
print(s.recv(2048))
s.close()

b''


## 2.1.7 Port Scanner

In [22]:
import socket

host = '192.168.0.1'
portas = [21,22,23,80]
print('Portas Abertas')
for porta in portas:
    s = socket.socket(socket.AF_INET,socket.SOCK_STREAM)
    codigoRetorno = s.connect_ex((host,porta))
    s.close()
    if codigoRetorno == 0 :
        print(porta)

Portas Abertas
80


## 2.1.8 Integrando Python com Nmap

In [23]:
import nmap
scanNmap = nmap.PortScanner()
scanNmap.scan("192.168.0.1","22,80")
print("Valores de scanNmap['192.168.0.1']")
print(scanNmap["192.168.0.1"])
print("-" * 40)

print("Chaves do dicionário scanNmap['192.168.0.1']")
for valores in scanNmap['192.168.0.1']:
    print(valores)
print('-' * 40)

print("Valores de scanNmap['192.168.0.1]['status']")
print(scanNmap['192.168.0.1']['status'])
print('-' * 40)

print("Valor de scanNmap['192.168.0.1']['status']['state']")
print(scanNmap['192.168.0.1']['status']['state'])

Valores de scanNmap['192.168.0.1']
{'hostnames': [{'name': '', 'type': ''}], 'addresses': {'ipv4': '192.168.0.1', 'mac': '48:00:33:05:B5:A0'}, 'vendor': {'48:00:33:05:B5:A0': 'Technicolor CH USA'}, 'status': {'state': 'up', 'reason': 'arp-response'}, 'tcp': {22: {'state': 'filtered', 'reason': 'no-response', 'name': 'ssh', 'product': '', 'version': '', 'extrainfo': '', 'conf': '3', 'cpe': ''}, 80: {'state': 'open', 'reason': 'syn-ack', 'name': 'http', 'product': 'micro_httpd', 'version': '', 'extrainfo': '', 'conf': '10', 'cpe': 'cpe:/a:acme:micro_httpd'}}}
----------------------------------------
Chaves do dicionário scanNmap['192.168.0.1']
hostnames
addresses
vendor
status
tcp
----------------------------------------
Valores de scanNmap['192.168.0.1]['status']
{'state': 'up', 'reason': 'arp-response'}
----------------------------------------
Valor de scanNmap['192.168.0.1']['status']['state']
up


In [30]:
import nmap
scanNmap = nmap.PortScanner()
scanNmap.scan('192.168.0.1','22,80','-v -sV')
print(scanNmap.command_line())
for host in scanNmap.all_hosts():
    print('Nmap scan report for host {}'.format(host))
    print('Host is {}'.format(scanNmap[host]["status"]["state"]))
    for protocolo in scanNmap[host].all_protocols():
        print("PORT\tSTATE\tSERVICE")
        for porta in scanNmap[host][protocolo]:
            alvo = scanNmap[host][protocolo][porta]
            print('{}/{}\t{}\t{}'.format(porta,protocolo,alvo["state"],alvo['name']))
    enderecoMAC = scanNmap[host]["addresses"]["mac"]
    print('MAC Address: {} ({})'.format(enderecoMAC,scanNmap[host]['vendor'][enderecoMAC]))

nmap -oX - -p 22,80 -v -sV 192.168.0.1
Nmap scan report for host 192.168.0.1
Host is up
PORT 	 STATE 	   SERVICE
22/tcp 	 filtered 	   ssh
80/tcp 	 open 	   http
MAC Address: 48:00:33:05:B5:A0 (Technicolor CH USA)


## 2.2 Exploração

### 2.2.1 Automatizando a execução de exploits

Através do uso da biblioteca `msfrpc` é possível acessar o Metasploit para automatizar o processo de exploração de vulnerabilidades. 

Para isso é necessário instalar a biblioteca no ambiente Python e configurar o metasploit para aceita os acesso remoto.

Para configurar o metasploit use o comando `load msgrpc Pass=<senha>`

In [ ]:
import time, msfrpc

cliente